## 1 Install library（ライブラリをインストール）

In [ ]:
import pandas as pd
df = pd.read_csv('ai4i2020.csv')
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


In [ ]:
!pip -q install smolagents
!pip -q install litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## 2 Make prompt（プロンプトの作成）

In [ ]:
prompt = "you are an awesome data analyst to perform Predictive Maintenance.Your task is to answer the question based on data analysis.\
          Please follow these instructions:\
          1. Review the data imported to ensure every feature is accounted for.\
          2. Data should be analysed into a logical, step-by-step order, using the specified prosesure and library.\
          3. You should import data from the directory '/content', do not use web search.\
          4. You should use pandas to extract data.\
          5. You should use HistGradientBoostingClassifier in sklearn to analyse data.\
          6. You should develop the predictive model to analyse data.\
          7. You should present the results reqired by users in tabular format.\
          8. Use the following format\
              - **Main actions are numbered** (e.g., 1, 2, 3).\
              - **result of each action shoud be presented in tabular format** \
          9. Results should be saved in 'output.csv' file.\
          "

## 3 Create agent with gemini-2.0-flash-thinking-exp-01-21（gemini-2.0-flash-thinking-exp-01-21を使いエージェント作成）

In [ ]:
from smolagents import CodeAgent, HfApiModel, LiteLLMModel

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-thinking-exp-01-21")
agent = CodeAgent(tools=[], model=model, add_base_tools=True, \
                  additional_authorized_imports=['pandas', 'math', 'queue', 'random', \
                 'time', 'sklearn', "matplotlib.pyplot"," seaborn", 'shap'], \
                  max_steps=12)


##  4 Predict the probability of Machine failure （故障確率の予測）

In [ ]:
%%time

question= " Could you provide the probability of Machine failure for each product by sklearn? Please follow this instruction.\
          1. Import file of 'ai4i2020.csv' from in the directory '/content'. You should use pandas to extract data. Do not use web search.\
          2. You should tell the 10 product ID who has the highest probability of Machine failure in test data, wchici is used for validation of model. \
          3. AUC metric should be provided to evaluate the model. \
          4. Devide 5 bins and provide the distrubution of product according to the probability of Machine failure ? \
          "
output=agent.run(
    prompt+question,
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ you are an awesome data analyst to perform Predictive Maintenance.Your task is to answer the question based on  │
│ data analysis.          Please follow these instructions:          1. Review the data imported to ensure every  │
│ feature is accounted for.          2. Data should be analysed into a logical, step-by-step order, using the     │
│ specified prosesure and library.          3. You should import data from the directory                          │
│ '/content/drive/MyDrive/01_Agent', do not use web search.          4. You should use pandas to extract data.    │
│ 5. You should use HistGradientBoostingClassifier in sklearn to analyse data.          6. You should develop the │
│ predictive model to analyse data.          7. You should present the results reqired by users in tabular        │
│ format.          8. Use the following format              - **Main actions are numbered** (e.g., 1, 2, 3).      │
│ - **result of each action shoud be presented in tabular format**           9. Results should be saved in        │
│ 'output.csv' file.           Could you provide the probability of Machine failure for each product by sklearn?  │
│ Please follow this instruction.          1. Import file of 'ai4i2020.csv' from in the directory                 │
│ '/content/drive/MyDrive/01_Agent'. You should use pandas to extract data. Do not use web search.          2.    │
│ You should tell the 10 product ID who has the highest probability of Machine failure in test data, wchici is    │
│ used for valisation of model.           3. AUC metric should be provided to evaluate the model.           4.    │
│ Devide 5 bins and provide the distrubution of customer according to the probability of Machine failure ?        │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-thinking-exp-01-21 ─────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  try:                                                                                                             
      df = pd.read_csv('/content/drive/MyDrive/01_Agent/ai4i2020.csv')                                             
      print("Data imported successfully.")                                                                         
      print(df.head())                                                                                             
  except FileNotFoundError:                                                                                        
      print("Error: File not found at '/content/drive/MyDrive/01_Agent/ai4i2020.csv'")                             
  except Exception as e:                                                                                           
      print(f"An error occurred: {e}")                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Data imported successfully.
   UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   
1    2     L47181    L                298.2                    308.7   
2    3     L47182    L                298.1                    308.5   
3    4     L47183    L                298.2                    308.6   
4    5     L47184    L                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  \
0                    1551         42.8                0                0    0   
1                    1408         46.3                3                0    0   
2                    1498         49.4                5                0    0   
3                    1433         39.5                7                0    0   
4                    1408         40.0                9                0    0   

   HDF  PWF  OSF  RNF  
0    0    0    0    0  
1    0    0    0    0  
2    0    0    0    0  
3    0    0    0    0  
4    0    0    0    0  

Out: None

[Step 0: Duration 19.26 seconds| Input tokens: 2,675 | Output tokens: 156]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("All results are saved to CSV files in '/content/drive/MyDrive/01_Agent/' directory, including top  
  10 product IDs with highest failure probability, AUC score, and probability bin distribution.")                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: All results are saved to CSV files in '/content/drive/MyDrive/01_Agent/' directory, including 
top 10 product IDs with highest failure probability, AUC score, and probability bin distribution.

[Step 11: Duration 2.37 seconds| Input tokens: 83,817 | Output tokens: 6,197]

CPU times: user 3.32 s, sys: 80.8 ms, total: 3.4 s
Wall time: 60 s


In [ ]:
print(output)

All results are saved to CSV files in '/content/drive/MyDrive/01_Agent/' directory, including top 10 product IDs with highest failure probability, AUC score, and probability bin distribution.


In [ ]:
!ls

ai4i2020.csv	     bank-additional-full.csv	output_20240125.csv  train.csv
auc_score.csv	     bank-additional-names.txt	test.csv	     UCI_Credit_Card.csv
bank-additional.csv  bin_distribution.csv	top_10_products.csv


In [ ]:
import pandas as pd
df = pd.read_csv('top_10_products.csv')
df

,Rank,Product ID,Probability_Machine_Failure
0,0,L53428,0.999972
1,1,L53939,0.999972
2,2,L57154,0.999971
3,3,L47428,0.999971
4,4,L56264,0.999971
5,5,L49013,0.999971
6,6,L48676,0.999971
7,7,M23706,0.999971
8,8,L50530,0.999971
9,9,L50864,0.999971


In [ ]:
import pandas as pd
df = pd.read_csv('bin_distribution.csv')
df

,Bin Rank,Probability Bin,Number of Products
0,0,Bin 1,1941
1,1,Bin 2,0
2,2,Bin 3,0
3,3,Bin 4,0
4,4,Bin 5,59
